### Código para obtener los centros de distribución de Soriana, Chedraui y Walmart

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import googlemaps
import folium

In [3]:
#urls de las páginas
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
url={"Soriana": "https://www.organizacionsoriana.com/centros_de_distribucion.html",
        "Chedraui": "https://www.grupochedraui.com.mx/en/division_logistica/index.html"}

#### Función general

In [4]:
def get_html(url, encoding=None, verify=False):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers, verify=verify)
    if encoding is not None:
        response.encoding = encoding
    return BeautifulSoup(response.text, "html.parser")


##### Soriana

In [5]:
soriana=get_html(url["Soriana"], "utf-8")

c:\Users\claud\miniconda3\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.organizacionsoriana.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
#Obtener todo lo de la tabla
tabla=soriana.find_all("table", {"class": "table table-striped"})


In [7]:
all_tables_data = []  # Lista vacía

for table in tabla:  # Iterar sobre cada tabla
    headers = [th.get_text().strip() for th in table.find_all('th')]  # Obtener headers
    table_data = []  # Lista vacía para almacenar los datos de la tabla
    rows = table.find_all('tr')  # Obtener todas las filas de la tabla
    
    for row in rows:  # iterar sobre cada fila
        cells = row.find_all('td')  # Obtener las celdas de la fila
        if cells:  # Si hay celdas en la fila
            row_data = [cell.get_text().strip() for cell in cells]  
            if row_data:  # Si hay datos en la fila
                table_data.append(dict(zip(headers, row_data)))  # Agregar los datos de la fila a la lista de datos de la tabla

    if table_data:  # Si hay datos en la tabla
        all_tables_data.append(table_data)  # Agregar los datos de la tabla a la lista de datos de todas las tablas

In [8]:
#Pasar la lista a un dataframe
soriana_df=pd.DataFrame()
for i in range(len(all_tables_data)):
    soriana_df=pd.concat([soriana_df, pd.DataFrame(all_tables_data[i])], ignore_index=True)
# Reordenar la df
df_reorganized = pd.DataFrame(columns=["nombre", "direccion", "tipo"])

rename_map = {
    "CENTROS DE DISTRIBUCIÓN FRESCOS": "Fresco",
    "CENTROS DE DISTRIBUCIÓN SECOS": "Seco",
    "CENTRO DE DISTRIBUCIÓN  E-COMMERCE": "E-Commerce"
}


for _, row in soriana_df.iterrows():
    
    for col_name, tipo in rename_map.items():
        if pd.notna(row[col_name]):
            df_reorganized =pd.concat([df_reorganized, pd.DataFrame([[row[col_name], row["DIRECCIÓN"], tipo]], columns=["nombre", "direccion", "tipo"])], ignore_index=True)
soriana_df=df_reorganized
#Añadir tienda
soriana_df["tienda"]="Soriana"
soriana_df

,nombre,direccion,tipo,tienda
0,CDF Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Fresco,Soriana
1,CDF Cuautitlán,"Autopista México-Querétaro km 36.8,C.P. 54700 ...",Fresco,Soriana
2,CDF Villahermosa,"Carretera Villahermosa-Cárdenas Km 164.1,Ranch...",Fresco,Soriana
3,CDF Guadalajara,"Periférico Sur Manuel Gómez Morín No. 5890,Col...",Fresco,Soriana
4,CDF Tijuana,Prolongación Club de Leones 10535.Fraccionamie...,Fresco,Soriana
5,CDF Hermosillo,Av. Chetumal entre Guatemala y Belice No. 1121...,Fresco,Soriana
6,CDS Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Seco,Soriana
7,CDS Tultitlán,"Km. 14.5 Carretera Puente de Vigas Cuautitlán,...",Seco,Soriana
8,CDS Querétaro,"Autopista México-Querétaro Km 186 S/N, Col Ent...",Seco,Soriana
9,CDS Guadalajara,"Periférico Sur Manuel Gómez Morín #5890, Col. ...",Seco,Soriana


##### Chedraui

In [9]:
chedraui=get_html(url["Chedraui"],encoding="utf-8",verify=False)

c:\Users\claud\miniconda3\lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.grupochedraui.com.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [10]:
#Obtener resultados
results=chedraui.find_all("div", {"class": "col-lg-12"})

#Obtener los nombres de los centros de distribución
nombres=chedraui.find_all("h2", {"class": "font-bold font-color-naranja font-size-18"})
direcciones=chedraui.find_all("ul", {"class": "ventajas-ul"})
#Obtener las direcciones
direcciones=[direccion.get_text().strip() for direccion in direcciones]
#Eliminar los saltos de línea y los espacios
direcciones=[direccion.replace("\n", "").replace("  ", "").replace("\t", " ") for direccion in direcciones]

#Pasar a un dataframe
chedraui_df=pd.DataFrame({"nombre": [nombre.get_text().strip() for nombre in nombres],
                          "direccion": direcciones})

chedraui_df["tienda"]="Chedraui"
chedraui_df

,nombre,direccion,tienda
0,Centro de Distribución Cancún,"Carr. Cancún - Puerto Morelos, Km. 329.5 Cancú...",Chedraui
1,Centro de Distribución Guadalajara,"Carretera El Verde, El Castillo No. 2000, Col....",Chedraui
2,Centro de Distribución La Paz,"Colima S/N, Col. Pueblo Nuevo, La Paz Baja Cal...",Chedraui
3,Centro de Distribución Monterrey,"Jaime Nuno No. 3660, Col. Del Norte, Monterrey...",Chedraui
4,Centro de Distribución Teoloyucan,"Carretera Cuautitlán-Zumpango Lt. 28 Mz. 825, ...",Chedraui
5,Centro de Distribución Veracruz,"Carretera Veracruz-Cardel, Calle tres zapotes ...",Chedraui
6,Centro de Distribución Villahermosa,"Carretera Villahermosa-Macuspana Km. 12.5, Cor...",Chedraui
7,Centro de Distribución Tampico,"Av. Hidalgo No. 501, Col. Del Pueblo, Tampico,...",Chedraui


In [11]:
#Pegar los dos dataframes
df=pd.concat([soriana_df, chedraui_df], ignore_index=True)
df

,nombre,direccion,tipo,tienda
0,CDF Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Fresco,Soriana
1,CDF Cuautitlán,"Autopista México-Querétaro km 36.8,C.P. 54700 ...",Fresco,Soriana
2,CDF Villahermosa,"Carretera Villahermosa-Cárdenas Km 164.1,Ranch...",Fresco,Soriana
3,CDF Guadalajara,"Periférico Sur Manuel Gómez Morín No. 5890,Col...",Fresco,Soriana
4,CDF Tijuana,Prolongación Club de Leones 10535.Fraccionamie...,Fresco,Soriana
5,CDF Hermosillo,Av. Chetumal entre Guatemala y Belice No. 1121...,Fresco,Soriana
6,CDS Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Seco,Soriana
7,CDS Tultitlán,"Km. 14.5 Carretera Puente de Vigas Cuautitlán,...",Seco,Soriana
8,CDS Querétaro,"Autopista México-Querétaro Km 186 S/N, Col Ent...",Seco,Soriana
9,CDS Guadalajara,"Periférico Sur Manuel Gómez Morín #5890, Col. ...",Seco,Soriana


#### Geocodear

In [12]:
#Credenciales de Google Maps API desde txt. Se debe crear un archivo de texto con las credenciales. Esto es personal.
with open("C:/Users/claud/Documents/credencialesgmaps.txt", "r") as file:
    api_key=file.read().strip()

gmaps=googlemaps.Client(key=api_key)

In [22]:
#Geocodificar las direcciones y ponerlo en la columna de coordenadas
df["coordenadas"]=df["direccion"].apply(lambda x: gmaps.geocode(x)[0]["geometry"]["location"])
#Crear dos columnas con las coordenadas
df["lat"]=df["coordenadas"].apply(lambda x: x["lat"])
df["lon"]=df["coordenadas"].apply(lambda x: x["lng"])
#Dropear la columna de coordenadas
df=df.drop(columns=["coordenadas"])
#Salvar el dataframe como excel
df.to_excel("cedis.xlsx", index=False)
df

,nombre,direccion,tipo,tienda,lat,lon
0,CDF Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Fresco,Soriana,25.892504,-100.261048
1,CDF Cuautitlán,"Autopista México-Querétaro km 36.8,C.P. 54700 ...",Fresco,Soriana,19.635243,-99.194294
2,CDF Villahermosa,"Carretera Villahermosa-Cárdenas Km 164.1,Ranch...",Fresco,Soriana,17.974040,-93.130237
3,CDF Guadalajara,"Periférico Sur Manuel Gómez Morín No. 5890,Col...",Fresco,Soriana,20.585621,-103.330130
4,CDF Tijuana,Prolongación Club de Leones 10535.Fraccionamie...,Fresco,Soriana,32.445086,-117.006424
5,CDF Hermosillo,Av. Chetumal entre Guatemala y Belice No. 1121...,Fresco,Soriana,15.783471,-90.230759
6,CDS Salinas Victoria,Carretera Salinas-Victoria km.5.5 Col. Satélit...,Seco,Soriana,25.892504,-100.261048
7,CDS Tultitlán,"Km. 14.5 Carretera Puente de Vigas Cuautitlán,...",Seco,Soriana,19.641365,-99.182595
8,CDS Querétaro,"Autopista México-Querétaro Km 186 S/N, Col Ent...",Seco,Soriana,20.501044,-100.139517
9,CDS Guadalajara,"Periférico Sur Manuel Gómez Morín #5890, Col. ...",Seco,Soriana,20.585621,-103.330130


In [27]:
mapa = folium.Map(location=[19.432608, -99.133209], zoom_start=6, tiles="http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}", attr="Google Satellite")

for _, row in df.iterrows():
    # Tooltip
    tooltip_text = f"<b>Nombre:</b> {row['nombre']}<br><b>Dirección:</b> {row['direccion']}<br><b>Tipo:</b> {row['tipo']}<br><b>Tienda:</b> {row['tienda']}"
    # marker
    folium.Marker(
        [row["lat"], row["lon"]],
        tooltip=tooltip_text
    ).add_to(mapa)

# URL del archivo
file_url = 'https://github.com/labdatos-se/cedis_supermercados/raw/main/cedis.xlsx'


html = f"""
<div style="position: fixed; bottom: 50px; left: 80%; z-index: 9999; background-color: white; padding: 10px; 
     border: 1px solid black; opacity: 0.8; font-size: 12px;">
    <a href="{file_url}" target="_blank" style="text-decoration: none; color: black;">Descargar listado de CEDIS</a>
</div>
"""
# Create a new map with the custom HTML
mapa.get_root().html.add_child(folium.Element(html))
# mapa
mapa

In [16]:
#Salvar mapa
mapa.save("cedis.html")
